<a href="https://colab.research.google.com/github/Precious3ita/Top-spotify-listening/blob/main/RecommendationSystem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [23]:
df1=pd.read_csv('/content/tmdb_5000_credits.csv (1).zip')

In [24]:
df2=pd.read_csv('/content/tmdb_5000_movies.csv (1).zip')

In [25]:
df1.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [27]:
df2.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [28]:
df1.columns = ['id','tittle','cast','crew']
df2= df2.merge(df1,on='id')

In [29]:
df2.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


Content Based Filtering
For this type of recommender system, the overview, cast, crew, keyword, tagline etc (content of dataset) is used to find similarity with other movies.

In [48]:
df2['production_companies'].head(5)

,production_companies
0,"[{'name': 'Ingenious Film Partners', 'id': 289..."
1,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'..."
2,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam..."
3,"[{'name': 'Legendary Pictures', 'id': 923}, {'..."
4,"[{'name': 'Walt Disney Pictures', 'id': 2}]"


For each cast, I will find its Term Frequency-Inverse Document Frequency with scikit-learn, read more about it here.

This will give you a matrix where each column represents a word in the cast vocabulary (all the words that appear in at least one document) and each row represents a movie, as before.This is done to reduce the importance of words that occur frequently in cast and therefore, their significance in computing the final similarity score.

In [31]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df2['cast'] = df2['cast'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['cast'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(4803, 218907)

Compute a similarity score with the cosine similarity scores.

Since we have used the TF-IDF vectorizer, calculating the dot product will directly give us the cosine similarity score. Therefore, we will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.

In [32]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [33]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()

In [34]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df2['title'].iloc[movie_indices]


In [40]:
get_recommendations('Superman')

,title
25,Titanic
2418,Dickie Roberts: Former Child Star
4216,Mr. Smith Goes to Washington
365,Contact
1847,GoodFellas
361,You Don't Mess with the Zohan
1432,Hot Pursuit
9,Batman v Superman: Dawn of Justice
1421,Hoffa
870,Superman II


In [41]:
get_recommendations('Superman II')

,title
813,Superman
1847,GoodFellas
361,You Don't Mess with the Zohan
2418,Dickie Roberts: Former Child Star
365,Contact
1432,Hot Pursuit
9,Batman v Superman: Dawn of Justice
4216,Mr. Smith Goes to Washington
25,Titanic
1421,Hoffa


If you notice, superman II and superman recommend one another since there is possibility of contenting the same cast in the same movie.

The quality of this recommender can be improved by adding more features. So from the cast, crew and keywords features, we need to extract the three most important actors, the director and the keywords associated with that movie. First we convert the dataset into a usable data


In [45]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['production_companies', 'crew', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

Function to extract the required information from each feature.

In [52]:
# Get the  name Ingenious Film Partners from the production companies feature. If director is not listed, return NaN
def get_production_companies(x):
    for i in x:
        if i['production_companies'] == 'Ingenious Film Partners':
            return i['name']
    return np.nan

In [51]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [56]:
# Get the name Ingenious Film Partners from the production companies feature. If not found, return NaN
def get_production_companies_name(x):
    if isinstance(x, list):
        for i in x:
            if isinstance(i, dict) and i.get('name') == 'Ingenious Film Partners':
                return i['name']
    return np.nan

# Apply the function to the production_companies column
df2['Ingenious Film Partners'] = df2['production_companies'].apply(get_production_companies_name)

features = ['cast', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [58]:
# Print the new features of the first 3 films
df2[['title', 'cast', 'production_companies', 'keywords', 'genres']].head(3)

,title,cast,production_companies,keywords,genres
0,Avatar,[],"[{'name': 'Ingenious Film Partners', 'id': 289...","[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,[],"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,[],"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[spy, based on novel, secret agent]","[Action, Adventure, Crime]"


In [61]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        cleaned_list = []
        for item in x:
            if isinstance(item, dict) and 'name' in item:
                cleaned_list.append(str.lower(item['name'].replace(" ", "")))
            elif isinstance(item, str):
                cleaned_list.append(str.lower(item.replace(" ", "")))
        return cleaned_list
    else:
        #Check if production companies exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [62]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'production_companies', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [64]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + ' '.join(x['production_companies']) + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)

In [65]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])

In [66]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [67]:
# Reset index of our main DataFrame and construct reverse mapping as before
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

We can now reuse our get_recommendations() function by passing in the new cosine_sim2 matrix as your second argument.

In [68]:
get_recommendations('Man of Steel', cosine_sim2)

,title
10,Superman Returns
9,Batman v Superman: Dawn of Justice
41,Green Lantern
813,Superman
870,Superman II
65,The Dark Knight
3,The Dark Knight Rises
119,Batman Begins
547,Sucker Punch
1296,Superman III


In [69]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# cosine similarity matrix (movies x movies)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Example: Recommend top 5 similar movies for a given one
movie_index = 0
similar_movies = cosine_sim[movie_index].argsort()[-6:-1][::-1]

print("Top 5 similar movies:", df2['title'].iloc[similar_movies].values)


Top 5 similar movies: ['American Gangster' 'The Dark Knight' 'Spider-Man 3'
 'The Life of David Gale' 'Inherent Vice']
